# 거리뷰 이미지를 이용한 불법 주정차 분석

## 1) 거리뷰 이미지 수집

### 1. 구글 street view API 사용
- 거리뷰 이미지를 url로 바로 받아올 수 있음
- 단점: 상세한 거리뷰 이미지가 없음

In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import time

load_dotenv()  
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_API_KEY = "발급받은 API 키"

def check_streetview_available(lat, lng):
    """
    왜 메타데이터 API를 먼저 쓰냐:
    이미지 API는 없는 곳에 요청해도 과금될 수 있지만
    메타데이터 API는 무료이고 이미지 존재 여부를 먼저 확인할 수 있음
    """
    url = (
        f"https://maps.googleapis.com/maps/api/streetview/metadata"
        f"?location={lat},{lng}"
        f"&key={GOOGLE_API_KEY}"
    )
    res = requests.get(url)
    data = res.json()

    # status가 OK면 이미지 있음, ZERO_RESULTS면 없음
    return data.get("status") == "OK"

def fetch_streetview(lat, lng, zone_id, save_dir="streetview_images"):
    os.makedirs(save_dir, exist_ok=True)
    headings = [0, 90, 180, 270]
    saved_paths = []

    for heading in headings:
        url = (
            f"https://maps.googleapis.com/maps/api/streetview"
            f"?size=640x480"
            f"&location={lat},{lng}"
            f"&heading={heading}"
            f"&fov=90"
            f"&key={GOOGLE_API_KEY}"
        )
        res = requests.get(url)
        if res.status_code == 200:
            path = f"{save_dir}/{zone_id}_h{heading}.jpg"
            with open(path, 'wb') as f:
                f.write(res.content)
            saved_paths.append(path)
        time.sleep(0.2)

    return saved_paths

# 실행
safezone_df = pd.read_csv('./어린이보호구역_위치정보.csv')

available = []
unavailable = []

for idx, row in safezone_df.iterrows():
    lat, lng = row['위도'], row['경도']

    if check_streetview_available(lat, lng):
        paths = fetch_streetview(lat, lng, zone_id=idx)
        available.append(row['대상시설명'])
        print(f"✅ [{idx}] {row['대상시설명']} → {len(paths)}장 수집")
    else:
        unavailable.append(row['대상시설명'])
        print(f"❌ [{idx}] {row['대상시설명']} → 이미지 없음 스킵")

print(f"\n수집 성공: {len(available)}개 / 스킵: {len(unavailable)}개")
print(f"\n이미지 없는 구역: {unavailable}")

❌ [0] 배성유치원 → 이미지 없음 스킵
❌ [1] 이솔유치원 → 이미지 없음 스킵
❌ [2] 성모 유치원 → 이미지 없음 스킵
❌ [3] 판교샘유치원 → 이미지 없음 스킵
❌ [4] 건영장안유치원 → 이미지 없음 스킵
❌ [5] 뽀뽀뽀유치원 → 이미지 없음 스킵
❌ [6] 해나유치원 → 이미지 없음 스킵
❌ [7] 샛별유치원 → 이미지 없음 스킵
❌ [8] 성마르코유치원 → 이미지 없음 스킵
❌ [9] 숲리라유치원 → 이미지 없음 스킵
❌ [10] 즐거운유치원 → 이미지 없음 스킵
❌ [11] 혜성유치원 → 이미지 없음 스킵
❌ [12] 산성어린이집 → 이미지 없음 스킵
❌ [13] 성남어린이집 → 이미지 없음 스킵
❌ [14] 성현어린이집 → 이미지 없음 스킵
❌ [15] 리플플러스어린이집 → 이미지 없음 스킵
❌ [16] 당촌초등학교 → 이미지 없음 스킵
❌ [17] 수내초등학교 → 이미지 없음 스킵
❌ [18] 산성3어린이집 → 이미지 없음 스킵
❌ [19] 은서유치원 → 이미지 없음 스킵
❌ [20] 예원유치원 → 이미지 없음 스킵
❌ [21] 다솜유치원 → 이미지 없음 스킵
❌ [22] 아름다운유치원 → 이미지 없음 스킵
❌ [23] 세화유치원 → 이미지 없음 스킵
❌ [24] 분당중앙유치원 → 이미지 없음 스킵
❌ [25] 보듬이나눔이어린이집 → 이미지 없음 스킵
❌ [26] 운중하나어린이집 → 이미지 없음 스킵
❌ [27] 푸르니이매어린이집 → 이미지 없음 스킵
❌ [28] 성남혜은학교 → 이미지 없음 스킵
❌ [29] 성은특수학교 → 이미지 없음 스킵
❌ [30] 케이디엘피어학원 → 이미지 없음 스킵
❌ [31] 판교초등학교 → 이미지 없음 스킵
❌ [32] 성남동초등학교 → 이미지 없음 스킵
❌ [33] 태평초등학교 → 이미지 없음 스킵
❌ [34] 성수초등학교 → 이미지 없음 스킵
❌ [35] 금빛초등학교 → 이미지 없음 스킵
❌ [36] 왕남초등학교 → 이미지 없음 스킵
❌ [37] 검단초등학교 → 이미지 없음 스킵
❌ [38] 불곡초등

### 2. 카카오맵 API + Selenium을 통해 직접 수집

In [2]:
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import pandas as pd

SAVE_DIR = "./roadview_images"
os.makedirs(SAVE_DIR, exist_ok=True)

def get_kakao_roadview(lat, lng, facility_name):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--window-size=1280,720")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )

    try:
        url = f"https://map.kakao.com/link/roadview/{lat},{lng}"
        driver.get(url)
        time.sleep(4)

        if "준비 중" in driver.page_source:
            print(f"  ❌ [{facility_name}] 로드뷰 없음")
            return None

        # 파일명에 시설명 사용
        # 왜 replace('/', '_')냐: 시설명에 '/'가 있으면 경로로 인식돼서 오류 발생하기 때문
        safe_name = facility_name.replace('/', '_').replace(' ', '_')
        path = f"{SAVE_DIR}/{safe_name}_북쪽.png"
        driver.save_screenshot(path)
        print(f"  ✅ [{facility_name}] 저장 완료 → {path}")
        return path

    except Exception as e:
        print(f"  ❌ [{facility_name}] 오류: {e}")
        return None

    finally:
        driver.quit()


safezone_df = pd.read_csv('../../Preprocessing/외부안전위험요소/어린이보호구역_위치정보.csv')

for idx, row in safezone_df.iterrows():
    print(f"[{idx}] {row['대상시설명']} 수집 중...")
    get_kakao_roadview(row['위도'], row['경도'], facility_name=row['대상시설명'])

[0] 배성유치원 수집 중...
  ✅ [배성유치원] 저장 완료 → ./roadview_images/배성유치원_북쪽.png
[1] 이솔유치원 수집 중...
  ✅ [이솔유치원] 저장 완료 → ./roadview_images/이솔유치원_북쪽.png
[2] 성모 유치원 수집 중...
  ✅ [성모 유치원] 저장 완료 → ./roadview_images/성모_유치원_북쪽.png
[3] 판교샘유치원 수집 중...
  ✅ [판교샘유치원] 저장 완료 → ./roadview_images/판교샘유치원_북쪽.png
[4] 건영장안유치원 수집 중...
  ✅ [건영장안유치원] 저장 완료 → ./roadview_images/건영장안유치원_북쪽.png
[5] 뽀뽀뽀유치원 수집 중...
  ✅ [뽀뽀뽀유치원] 저장 완료 → ./roadview_images/뽀뽀뽀유치원_북쪽.png
[6] 해나유치원 수집 중...
  ✅ [해나유치원] 저장 완료 → ./roadview_images/해나유치원_북쪽.png
[7] 샛별유치원 수집 중...
  ✅ [샛별유치원] 저장 완료 → ./roadview_images/샛별유치원_북쪽.png
[8] 성마르코유치원 수집 중...
  ✅ [성마르코유치원] 저장 완료 → ./roadview_images/성마르코유치원_북쪽.png
[9] 숲리라유치원 수집 중...
  ✅ [숲리라유치원] 저장 완료 → ./roadview_images/숲리라유치원_북쪽.png
[10] 즐거운유치원 수집 중...
  ✅ [즐거운유치원] 저장 완료 → ./roadview_images/즐거운유치원_북쪽.png
[11] 혜성유치원 수집 중...
  ✅ [혜성유치원] 저장 완료 → ./roadview_images/혜성유치원_북쪽.png
[12] 산성어린이집 수집 중...
  ✅ [산성어린이집] 저장 완료 → ./roadview_images/산성어린이집_북쪽.png
[13] 성남어린이집 수집 중...
  ✅ [성남어린이집] 저장 완료 → ./roadview_images/성남어린이집_북쪽.p